# Import Socorro crash data into the Data Platform

We want to be able to store Socorro crash data in Parquet form so that it can be made accessible from re:dash.

See [Bug 1273657](https://bugzilla.mozilla.org/show_bug.cgi?id=1273657) for more details

In [ ]:
import logging
log = logging.getLogger(__name__)
log.setLevel(logging.INFO)

We create the pyspark datatype for representing the crash data in spark. This is a slightly modified version of [peterbe/crash-report-struct-code](https://github.com/peterbe/crash-report-struct-code).  

In [ ]:
from pyspark.sql.types import *

def create_struct(schema):
    """ Take a JSON schema and return a pyspark StructType of equivalent structure. """
    
    replace_definitions(schema, schema['definitions'])
    assert '$ref' not in str(schema), 're-write didnt work'
    
    struct = StructType()
    for row in get_rows(schema):
        struct.add(row)

    return struct

def replace_definitions(schema, definitions):
    """ Replace references in the JSON schema with their definitions."""

    if 'properties' in schema:
        for prop, meta in schema['properties'].items():
            replace_definitions(meta, definitions)
    elif 'items' in schema:
        if '$ref' in schema['items']:
            ref = schema['items']['$ref'].split('/')[-1]
            schema['items'] = definitions[ref]
            replace_definitions(schema['items'], definitions)
        else:
            replace_definitions(schema['items'], definitions)
    elif '$ref' in str(schema):
        log.error("Reference not found for schema: {}".format(str(schema)))
        raise Exception

def get_rows(schema):
    """ Map the fields in a JSON schema to corresponding data structures in pyspark."""
    
    if 'properties' not in schema:
        log.error("Invalid JSON schema: properties field is missing.")
        raise Exception
        
    for prop in sorted(schema['properties']):
        meta = schema['properties'][prop]
        if 'string' in meta['type']:
            logging.debug("{!r} allows the type to be String AND Integer".format(prop))
            yield StructField(prop, StringType(), 'null' in meta['type'])
        elif 'integer' in meta['type']:
            yield StructField(prop, IntegerType(), 'null' in meta['type'])
        elif 'boolean' in meta['type']:
            yield StructField(prop, BooleanType(), 'null' in meta['type'])
        elif meta['type'] == 'array' and 'items' not in meta:
            # Assuming strings in the array
            yield StructField(prop, ArrayType(StringType(), False), True)
        elif meta['type'] == 'array' and 'items' in meta:
            struct = StructType()
            for row in get_rows(meta['items']):
                struct.add(row)
            yield StructField(prop, ArrayType(struct), True)
        elif meta['type'] == 'object':
            struct = StructType()
            for row in get_rows(meta):
                struct.add(row)
            yield StructField(prop, struct, True)
        else:
            log.error("Invalid JSON schema: {}".format(str(meta)[:100]))
            raise Exception

First fetch from the primary source in s3 as per [bug 1312006](https://bugzilla.mozilla.org/show_bug.cgi?id=1312006). We fall back to the github location if this is not available.

In [ ]:
import urllib2
import json

primary_url = "todo//:primary"
fallback_url = "https://raw.githubusercontent.com/mozilla/socorro/master/socorro/schemas/crash_report.json"

try:
    log.info("Fetching latest crash data schema from {}"
             .format(primary_url))
    resp = urllib2.urlopen(primary_url)
except Exception as e:
    log.warning(("Could not fetch schema from {}: {}\n"
                 "Fetching crash data schema from {}")
                .format(primary_url, e, fallback_url))
    resp = urllib2.urlopen(fallback_url)

data = json.load(resp)

### Read crash data as json, convert it to parquet

In [ ]:
crash_schema = create_struct(data)

In [ ]:
from datetime import datetime as dt, timedelta, date

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days) + 1):
        yield (end_date - timedelta(n)).strftime("%Y%m%d")

def import_day(d):
    source_s3path = "s3://org-allizom-telemetry-crashes/v1/crash_report"
    dest_s3path = "s3://net-mozaws-prod-us-west-2-pipeline-analysis/mreid/socorro_crash/v1"
    num_partitions = 5
    log.info("Processing {}, started at {}".format(d, dt.utcnow()))
    cur_source_s3path = "{}/{}".format(source_s3path, d)
    cur_dest_s3path = "{}/crash_date={}".format(dest_s3path, d)
    df = sqlContext.read.json(cur_source_s3path, schema=crash_schema)
    df.repartition(num_partitions).write.parquet(cur_dest_s3path, mode="overwrite")

def backfill(start_date_yyyymmdd):
    start_date = dt.strptime(start_date_yyyymmdd, "%Y%m%d")
    end_date = dt.utcnow() - timedelta(1) # yesterday
    for d in daterange(start_date, end_date):
        try:
            import_day(d)
        except Exception as e:
            log.error(e)

In [ ]:
yesterday = dt.strftime(dt.utcnow() - timedelta(1), "%Y%m%d")
import_day(yesterday)

#backfill("20160902")